# Adding transposed arrays with Dask and CuPy



## Use a DGX

In [ ]:
import dask
import dask.config

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster(diagnostics_port=9100)
client = Client(cluster)
client

## Create Random Dataset

This is done on GPU.

In [ ]:
import dask
import dask.array
import cupy


rs = dask.array.random.RandomState(RandomState=cupy.random.RandomState)
a = rs.random((10000, 10000), chunks=100)
a = a.persist()

## Add array with its transpose

This takes an array and adds it to its transpose. This involves some computations that have dependencies and some that don't.

In [ ]:
out = a + a.T

In [ ]:
import time
start = time.time()

In [ ]:
out = out.persist()
%time _ = wait(out)

In [ ]:
out.shape

In [ ]:
_ = client.profile(start=start, filename='dask-cupy-add-transpose.html')

## Inspect output

In [ ]:
out[:10, :10].compute()

In [ ]:
from distributed.utils import format_bytes

In [ ]:
format_bytes(out.nbytes)